Figure out how to get the data from all the snotel sites into one dataframe

Thinking averate? Or median?

In [46]:
import pandas as pd
# https://stackoverflow.com/questions/3207219/how-do-i-list-all-files-of-a-directory
from os import listdir
from os.path import isfile, join

# Get all the seasonal snotel reports
# could just use Stations Selected, but individual files might be easier

def get_seasonal_snotel_files():
    path = '../content/csv/'
    files = [f for f in listdir(path) if isfile(join('../content/csv/', f))]
    seasonal_snotel_files = [f for f in files if f.endswith('_Nov01_Apr30.csv')]
    seasonal_snotel_files.remove('stations_selected_Nov01_Apr30.csv')
    return seasonal_snotel_files

get_seasonal_snotel_files()

['Surprise_Lakes_WA_Nov01_Apr30.csv',
 'Corral_Pass_WA_Nov01_Apr30.csv',
 'Cougar_Mountain_WA_Nov01_Apr30.csv',
 'White_Pass_E.S_WA_Nov01_Apr30.csv',
 'Morse_Lake_WA_Nov01_Apr30.csv',
 'Skate_Creek_WA_Nov01_Apr30.csv',
 'Mowich_WA_Nov01_Apr30.csv',
 'Swift_Creek_WA_Nov01_Apr30.csv',
 'Sawmill_Ridge_WA_Nov01_Apr30.csv',
 'Spirit_Lake_WA_Nov01_Apr30.csv',
 'Lone_Pine_WA_Nov01_Apr30.csv',
 'Sheep_Canyon_WA_Nov01_Apr30.csv',
 'Potato_Hill_WA_Nov01_Apr30.csv',
 'Huckleberry_Creek_WA_Nov01_Apr30.csv']

Load them all into dataframes

In [47]:
def get_seasonal_snotel_dataframes():
    return [pd.read_csv(f'../content/csv/{f}', comment='#') for f in get_seasonal_snotel_files()]

get_seasonal_snotel_dataframes()[0].head()

,Date,Station Id,Station Name,Snow Water Equivalent (in),Snow Depth (in),Precipitation Accumulation (in),Air Temperature Observed (degF),Elevation (ft),Latitude,Longitude
0,2022-11-01 00:00,804,Surprise Lakes,0.0,0.0,6.5,34.5,4290,46.09497,-121.76345
1,2022-11-01 01:00,804,Surprise Lakes,-0.4,-2.0,6.5,34.3,4290,46.09497,-121.76345
2,2022-11-01 02:00,804,Surprise Lakes,-0.3,-3.0,6.5,34.3,4290,46.09497,-121.76345
3,2022-11-01 03:00,804,Surprise Lakes,-0.3,-3.0,6.6,34.2,4290,46.09497,-121.76345
4,2022-11-01 04:00,804,Surprise Lakes,-0.3,-3.0,6.6,34.2,4290,46.09497,-121.76345


Aggregate with average

In [55]:
def drop_non_numerical(snotel_df):
    """Drop columns that should not be part of the aggregation"""
    return snotel_df.drop(['Station Id', 'Station Name', 'Latitude', 'Longitude'], axis=1)

def set_up_dataframe_for_aggregation(dfs):
    # Make sure all the columns are the same
    assert all(len(dfs[0].columns.intersection(df.columns)) == dfs[0].shape[1] for df in dfs), \
        'All dataframes must have the same columns'   
    # Drop columns not wanted in aggregation
    dfs = [drop_non_numerical(df) for df in dfs]
    # Set the index of each dataframe to be the date
    dfs = [df.set_index('Date') for df in dfs]
    # Concatenate all the dataframes
    cat = pd.concat(dfs)
    # Group them by their index
    by_index = cat.groupby(cat.index)

    return by_index

def get_average_seasonal_snotel_dataframe(dfs=get_seasonal_snotel_dataframes()):
    dfs = set_up_dataframe_for_aggregation(dfs)
    means = dfs.mean()
    return means

# dfs = get_seasonal_snotel_dataframes()
# dfs[0] = dfs[0].drop('Date', axis=1)
# should fail   
# get_seasonal_snotel_dataframes(dfs)

get_average_seasonal_snotel_dataframe().iloc[200:205]

,Snow Water Equivalent (in),Snow Depth (in),Precipitation Accumulation (in),Air Temperature Observed (degF),Elevation (ft)
Date,,,,,
2022-11-09 08:00,2.607143,11.000000,13.478571,26.071429,4096.428571
2022-11-09 09:00,2.600000,10.923077,13.471429,28.064286,4096.428571
2022-11-09 10:00,2.642857,10.923077,13.514286,30.778571,4096.428571
2022-11-09 11:00,2.671429,10.538462,13.571429,32.435714,4096.428571
2022-11-09 12:00,2.707143,10.615385,13.600000,32.814286,4096.428571


Aggregate with median

In [56]:
def get_median_seasonal_snotel_dataframe(dfs=get_seasonal_snotel_dataframes()):
    dfs = set_up_dataframe_for_aggregation(dfs)
    meds = dfs.median()
    return meds

get_median_seasonal_snotel_dataframe().iloc[200:205]

,Snow Water Equivalent (in),Snow Depth (in),Precipitation Accumulation (in),Air Temperature Observed (degF),Elevation (ft)
Date,,,,,
2022-11-09 08:00,2.25,9.0,13.45,26.05,4140.0
2022-11-09 09:00,2.15,9.0,13.50,27.30,4140.0
2022-11-09 10:00,2.30,8.0,13.60,29.85,4140.0
2022-11-09 11:00,2.30,8.0,13.60,32.20,4140.0
2022-11-09 12:00,2.35,8.0,13.65,32.80,4140.0
